In [2]:
import sys

sys.path.insert(0, '../')

import graph_features
print(type(graph_features))
print(dir(graph_features))

<class 'module'>
['__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'extract_graph_features']


In [5]:
node_opcode, node_feat, edge_index, node_config_ids, node_splits = graph_features.extract_graph_features('../data/pb/layout/xla/default/train/brax_es.pb')

In [6]:
node_opcode

array([ 63,  63,   2, ...,  70,  59, 100], dtype=uint8)